# lab5 : Lab 5: Xây dựng mô hình RNN cho bài toán Nhận dạng
# Thực thể Tên (NER)

Tải dữ liệu từ huggingface 

In [1]:
from datasets import load_dataset
from torch.nn import Embedding

dataset = load_dataset("conll2003")
print(dataset)


/home/manh/code/nlp/.venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [2]:
train_sentences = dataset["train"]['tokens']
train_ner_tags = dataset['train']['ner_tags']
train_ner_names = dataset['train'].features['ner_tags'].feature.names
ner_dic = {i+1 :ner for (i , ner ) in enumerate(train_ner_names)}
ner_dic

{1: 'O',
 2: 'B-PER',
 3: 'I-PER',
 4: 'B-ORG',
 5: 'I-ORG',
 6: 'B-LOC',
 7: 'I-LOC',
 8: 'B-MISC',
 9: 'I-MISC'}

chuyển ner_tags ở dạng số nguyên về dạng tên nhãn . 

In [3]:
def convert_idx_to_name(example):
    example['ner_tags_str'] = [ner_dic[i + 1] for i in example['ner_tags']]
    return example
dataset = dataset.map(convert_idx_to_name)

In [30]:
ner_name = dataset['train']['ner_tags_str']


xây dựng từ điển 

In [5]:
word_idx = {"<UNK>"  : 1  , "<PAD>" : 0}
temp = 2
for words in train_sentences:
    # word_idx[word] = i+24
    for word in words:
        if word not in word_idx : 
            word_idx[word] = temp 
            temp+=1
print(len(word_idx))

23625


In [6]:
# xây dựng từ điển cho ner_tag 
ner_tag_idx  = {value : key for key , value in ner_dic.items()}
ner_tag_idx


{'O': 1,
 'B-PER': 2,
 'I-PER': 3,
 'B-ORG': 4,
 'I-ORG': 5,
 'B-LOC': 6,
 'I-LOC': 7,
 'B-MISC': 8,
 'I-MISC': 9}

In [7]:
ner_tag_idx

{'O': 1,
 'B-PER': 2,
 'I-PER': 3,
 'B-ORG': 4,
 'I-ORG': 5,
 'B-LOC': 6,
 'I-LOC': 7,
 'B-MISC': 8,
 'I-MISC': 9}

Task2 : Tạo pytorch và dataloader 

In [8]:
def transform_data(sentences , ner_tags):
    unk = word_idx.get("<UNK>" , 1)
    X = []
    Y = []
    for sentence , tag in zip(sentences , ner_tags):
        words  = []
        tags = []
        for word , i in zip(sentence , tag):
            if word in word_idx : 
                words.append(word_idx[word])
            else:
                words.append(unk)
            tags.append(ner_tag_idx[i])
        X.append(words)
        Y.append(tags)
    return X , Y  
        

In [9]:
import torch
from torch.utils.data import Dataset , DataLoader
class NERDataset(Dataset):
    def __init__(self , words , ner_tags ) : 
        self.words = words 
        self.ner_tags = ner_tags 
    def __len__(self):
        return len(self.ner_tags)
    def __getitem__(self , idx):
        return torch.tensor(self.words[idx] , dtype = torch.long), torch.tensor(self.ner_tags[idx] , dtype = torch.long)

add padding by pad_sequence


In [10]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    sentences = [i[0] for i in batch]
    tags = [i[1] for i in batch]
    padding_value = 0
    padding_sentence = pad_sequence(sentences , batch_first=True , padding_value=padding_value)
    padding_tag = pad_sequence(tags , batch_first = True , padding_value= 0)
    length = [len(i) for i in tags]
    return padding_sentence , padding_tag , length

Train data

In [11]:
X_train , Y_train = transform_data(dataset['train']['tokens'] , dataset['train']['ner_tags_str'])
train_dataset = NERDataset(X_train , Y_train)
train_loader = DataLoader(train_dataset , batch_size = 64 , collate_fn = collate_fn)


val dataset

In [12]:
X_val , Y_val = transform_data(dataset['validation']['tokens'] , dataset['validation']['ner_tags_str'])
val_dataset = NERDataset(X_val , Y_val)
val_loader = DataLoader(val_dataset , batch_size = 64 , collate_fn = collate_fn)

test loader

In [40]:
X_test , Y_test= transform_data(dataset['test']['tokens'] , dataset['test']['ner_tags_str'])
test_dataset = NERDataset(X_val , Y_val)
test_loader = DataLoader(test_dataset , batch_size = 64 , collate_fn = collate_fn)

Xây dựng model RNN 

In [13]:
import torch.nn as nn 
from torch.nn.utils.rnn import pack_padded_sequence , pad_packed_sequence
class SimpleRNN(nn.Module):
    def __init__(self ,vocab_size ,  embedding_dim , hidden_dim , output_size):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(num_embeddings = vocab_size , embedding_dim = embedding_dim , padding_idx=0)
        self.rnn = nn.RNN(input_size = embedding_dim , hidden_size = hidden_dim , num_layers = 1 , batch_first = True)
        self.fc = nn.Linear(embedding_dim , output_size)
    def forward(self  , x , length):
        x = self.embedding(x)
        packed = pack_padded_sequence(x , length , enforce_sorted=False , batch_first= True)
        paced_out , r_n = self.rnn(packed)
        pad_out , length = pad_packed_sequence(paced_out , batch_first= True)
        out = self.fc(pad_out)
        return out


BI LSTM

In [35]:
class BiLSTM(nn.Module):
    def __init__(self ,vocab_size , embedding_dim , hidden_dim , output_size):
        super(BiLSTM , self).__init__()
        self.embedding = nn.Embedding(num_embeddings = vocab_size , embedding_dim = embedding_dim , padding_idx=0 )
        self.lstm = nn.LSTM(
            input_size = embedding_dim ,
            hidden_size = hidden_dim ,
            num_layers = 1 ,
            batch_first = True,
            bidirectional = True
        )
        self.fc = nn.Linear(hidden_dim *2, output_size)
    def forward(self , x , length):
        x = self.embedding(x)
        packed = pack_padded_sequence(x , length , enforce_sorted=False , batch_first=True)
        out , (h , c) = self.lstm(packed)
        out_padded , length = pad_packed_sequence(out , batch_first= True)

        out = self.fc(out_padded)
        return out

fit method


In [14]:
from torch.utils.tensorboard import SummaryWriter

def fit(model ,train_loader, criterion , optimizer , device = "cuda" , epochs = 100):
    writer = SummaryWriter(log_dir = "runs/ner")
    model = model.to(device)
    for epoch in range(epochs):
        train_loss = 0.0
        train_acc = 0
        total = 0
        for x , y , length in train_loader :
            x , y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x , length)
            out_reshaped = out.view(-1 , out.shape[-1])
            y_reshape = y.view(-1)
            # cross entropy loss
            loss = criterion(out_reshaped, y_reshape)
            loss.backward()
            optimizer.step()
            # predict
            _ , predicted = torch.max(out , dim =  2 )
            mask = (y!=0)
            train_loss += loss.item()
            train_acc += ((predicted == y) & mask).sum().item()
            total += mask.sum().item()
        train_loss_avg = train_loss / len(train_loader)
        train_acc_avg = train_acc / total
        writer.add_scalar('loss/train' , train_loss_avg , epoch)
        writer.add_scalar('accuracy/train' , train_acc_avg , epoch)
        print(f"Train loss: {train_loss_avg :.4F} | Train accuracy: {train_acc_avg:.4F}")


2025-12-07 12:43:12.633055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-07 12:43:13.598926: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


train BiLstm

In [39]:
from torch.optim import Adam
modelBiLSTM = BiLSTM(vocab_size=23625 , embedding_dim=256 , hidden_dim=256 , output_size=10)
optimizer = Adam(modelBiLSTM.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)
fit(modelBiLSTM , train_loader, criterion , optimizer , epochs = 15)

Train loss: 0.6614 | Train accuracy: 0.8457
Train loss: 0.2856 | Train accuracy: 0.9177
Train loss: 0.1485 | Train accuracy: 0.9551
Train loss: 0.0752 | Train accuracy: 0.9775
Train loss: 0.0358 | Train accuracy: 0.9909
Train loss: 0.0165 | Train accuracy: 0.9967
Train loss: 0.0078 | Train accuracy: 0.9991
Train loss: 0.0038 | Train accuracy: 0.9997
Train loss: 0.0022 | Train accuracy: 0.9999
Train loss: 0.0014 | Train accuracy: 0.9999
Train loss: 0.0011 | Train accuracy: 0.9999
Train loss: 0.0008 | Train accuracy: 1.0000
Train loss: 0.0006 | Train accuracy: 1.0000
Train loss: 0.0005 | Train accuracy: 1.0000
Train loss: 0.0004 | Train accuracy: 1.0000


train simple RNN

In [48]:
from torch.optim import Adam
modelRNN = SimpleRNN(vocab_size=23625 , embedding_dim=256 , hidden_dim=256 , output_size=10)
optimizer = Adam(modelRNN.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)
fit(modelRNN , train_loader, criterion , optimizer , epochs=50)

Train loss: 0.7087 | Train accuracy: 0.8249
Train loss: 0.4079 | Train accuracy: 0.8853
Train loss: 0.2737 | Train accuracy: 0.9191
Train loss: 0.1892 | Train accuracy: 0.9424
Train loss: 0.1318 | Train accuracy: 0.9593
Train loss: 0.0914 | Train accuracy: 0.9719
Train loss: 0.0632 | Train accuracy: 0.9806
Train loss: 0.0443 | Train accuracy: 0.9867
Train loss: 0.0313 | Train accuracy: 0.9910
Train loss: 0.0226 | Train accuracy: 0.9937
Train loss: 0.0170 | Train accuracy: 0.9954
Train loss: 0.0136 | Train accuracy: 0.9964
Train loss: 0.0114 | Train accuracy: 0.9970
Train loss: 0.0101 | Train accuracy: 0.9974
Train loss: 0.0091 | Train accuracy: 0.9976
Train loss: 0.0086 | Train accuracy: 0.9977
Train loss: 0.0081 | Train accuracy: 0.9978
Train loss: 0.0077 | Train accuracy: 0.9978
Train loss: 0.0074 | Train accuracy: 0.9979
Train loss: 0.0072 | Train accuracy: 0.9979
Train loss: 0.0071 | Train accuracy: 0.9979
Train loss: 0.0068 | Train accuracy: 0.9979
Train loss: 0.0067 | Train accur

evaluate

In [44]:
# evaluate model
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
def evaluate(model , val_loader , criterion , device = "cuda" , epochs = 100):
    val_loss = 0.0
    val_acc = 0
    total = 0
    model.to(device)
    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        for x , y , length in val_loader :
            x , y = x.to(device), y.to(device)
            out = model(x , length)
            out_reshaped = out.view(-1 , out.shape[-1])
            y_reshape = y.view(-1)
            mask = (y!= 0)

            # predict
            _ , predicted = torch.max(out , dim = 2 )

            # loss , acc
            val_loss += criterion(out_reshaped, y_reshape).item()
            val_acc += ((y == predicted )& mask).sum().item()
            total += mask.sum().item()
            # adv evaluate
            for i in range(x.shape[0]):
                true_seq = []
                pred_seq = []
                for j in range(length[i]):
                    true_seq.append(ner_dic[y[i][j].item()])
                    pred_seq.append(ner_dic[predicted[i][j].item()])
                y_true.append(true_seq)
                y_pred.append(pred_seq)
        val_loss_avg = val_loss/len(val_loader)
        val_acc_avg = val_acc / total
        print(f" loss: {val_loss_avg:.4f} |  accuracy: {val_acc_avg:.4f}")
        print("Precision:", precision_score(y_true, y_pred))
        print("Recall:", recall_score(y_true, y_pred))
        print("F1-score:", f1_score(y_true, y_pred))
        print("\n", classification_report(y_true, y_pred))

RNN evaluate

In [52]:
evaluate(modelRNN , val_loader, criterion)
print("*" * 100  )
print("on test set")
evaluate(modelRNN , test_loader, criterion)

 loss: 0.2858 |  accuracy: 0.9419
Precision: 0.6565519323671497
Recall: 0.7319084483338943
F1-score: 0.6921852618176031

               precision    recall  f1-score   support

         LOC       0.57      0.82      0.67      1837
        MISC       0.71      0.70      0.71       922
         ORG       0.59      0.68      0.63      1341
         PER       0.83      0.70      0.76      1842

   micro avg       0.66      0.73      0.69      5942
   macro avg       0.68      0.72      0.69      5942
weighted avg       0.68      0.73      0.70      5942

****************************************************************************************************
on test set
 loss: 0.2858 |  accuracy: 0.9419
Precision: 0.6565519323671497
Recall: 0.7319084483338943
F1-score: 0.6921852618176031

               precision    recall  f1-score   support

         LOC       0.57      0.82      0.67      1837
        MISC       0.71      0.70      0.71       922
         ORG       0.59      0.68      0.63  

## Kết quả đánh giá mô hình NER theo RNN

| Chỉ số tổng quan | Giá trị |
|------------------|---------|
| Validation Loss  | 0.5635  |
| Accuracy (token) | 0.9171  |
| Precision        | 0.53798 |
| Recall           | 0.74739 |
| F1-score         | 0.62563 |

---

## Kết quả theo từng loại thực thể

| Entity | Precision | Recall | F1-score | Support |
|--------|-----------|--------|----------|---------|
| LOC    | 0.67      | 0.83   | 0.74     | 1837    |
| MISC   | 0.53      | 0.70   | 0.60     | 922     |
| ORG    | 0.32      | 0.77   | 0.45     | 1341    |
| PER    | 0.83      | 0.68   | 0.74     | 1842    |

---

## Trung bình

| Loại trung bình | Precision | Recall | F1-score | Support |
|------------------|-----------|--------|----------|---------|
| Micro avg        | 0.54      | 0.75   | 0.63     | 5942    |
| Macro avg        | 0.59      | 0.74   | 0.63     | 5942    |
| Weighted avg     | 0.62      | 0.75   | 0.65     | 5942    |


đánh giá mô hình BiLSTM

In [45]:
evaluate(modelBiLSTM , val_loader, criterion)
print("*" * 100  )
print("on test set")
evaluate(modelBiLSTM , test_loader, criterion)

 loss: 0.2223 |  accuracy: 0.9509
Precision: 0.7215370018975332
Recall: 0.767923258162235
F1-score: 0.7440078265123105

               precision    recall  f1-score   support

         LOC       0.67      0.87      0.76      1837
        MISC       0.82      0.70      0.76       922
         ORG       0.65      0.70      0.67      1341
         PER       0.81      0.75      0.78      1842

   micro avg       0.72      0.77      0.74      5942
   macro avg       0.74      0.75      0.74      5942
weighted avg       0.73      0.77      0.74      5942

****************************************************************************************************
on test set
 loss: 0.2223 |  accuracy: 0.9509
Precision: 0.7215370018975332
Recall: 0.767923258162235
F1-score: 0.7440078265123105

               precision    recall  f1-score   support

         LOC       0.67      0.87      0.76      1837
        MISC       0.82      0.70      0.76       922
         ORG       0.65      0.70      0.67    

# dự đoán NER của một câu bất

In [53]:
def predict_sentence(sentence, model):
    # 1. Tách từ
    tokens = sentence.split()

    # 2. Convert từ -> index
    # (Lưu ý: nên xử lý trường hợp từ lạ <UNK> nếu có, tránh lỗi key error)
    sentence_idx = [word_idx[w] if w in word_idx else 1 for w in tokens]

    # 3. Tạo tensor input (Dữ liệu X thì CẦN ở trên GPU/Device của model)
    # Lấy device tự động từ model để code linh hoạt hơn
    device = next(model.parameters()).device
    idx_tensor = torch.tensor(sentence_idx, dtype=torch.long).unsqueeze(0).to(device)

    # 4. Chiều dài (QUAN TRỌNG: Length PHẢI ở trên CPU)
    # --- SỬA Ở ĐÂY: Bỏ .to("cuda") hoặc force .to("cpu") ---
    length = torch.tensor([len(tokens)], dtype=torch.long).to("cpu")

    # 5. Predict
    model.eval()
    with torch.no_grad():
        out = model(idx_tensor, length)
        _, predicted = torch.max(out, dim=2)
    predicted = predicted.squeeze(0).tolist()
    return [ner_dic[i] for  i in predicted]



In [46]:
# Chạy thử RNN
sentences =["John lives in New York." , 'Barack Obama was born in Hawaii.' , 'The FIFA World Cup will be held in Canada.' ,'VNU University is located in Hanoi']
for sentence in sentences:
    y = predict_sentence(sentence, modelRNN)
    print(sentence  , y)

John lives in New York. ['B-PER', 'I-PER', 'O', 'B-LOC', 'I-LOC']
Barack Obama was born in Hawaii. ['O', 'O', 'O', 'O', 'O', 'B-LOC']
The FIFA World Cup will be held in Canada. ['O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O']
VNU University is located in Hanoi ['B-ORG', 'B-ORG', 'O', 'O', 'O', 'O']


In [47]:
# Chạy thử BiLSTM
sentences =["John lives in New York." , 'Barack Obama was born in Hawaii.' , 'The FIFA World Cup will be held in Canada.' ,'VNU University is located in Hanoi']
for sentence in sentences:
    y = predict_sentence(sentence, modelBiLSTM)
    print(sentence  , y)

John lives in New York. ['B-PER', 'O', 'O', 'B-LOC', 'I-LOC']
Barack Obama was born in Hawaii. ['B-PER', 'B-PER', 'O', 'O', 'O', 'B-LOC']
The FIFA World Cup will be held in Canada. ['O', 'B-MISC', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'B-LOC']
VNU University is located in Hanoi ['B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-LOC']
